### Load Overlay

In [36]:
from pynq import Overlay
from pynq import allocate
overlay = Overlay('/home/xilinx/pynq/overlays/matmul/matmul1024.bit')

In [37]:
overlay?

#### Create Driver for IP

In [38]:
matmul_ip = overlay.matmul_0
matmul_ip?

In [39]:
dma = overlay.axi_dma_0
dma?


In [40]:
matmul_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0)
}

In [41]:

#dma.register_map.MM2S_DMASR.idle=1
#dma.register_map
dma.sendchannel

In [42]:
import numpy as np
import time
a=np.ones((1024,1024),dtype=np.int32)
b=np.ones((1024,1024),dtype=np.int32)
start_time=time.time()
print("start")
c=np.dot(a,b)
stop_time=time.time()
sw_exec_time = stop_time-start_time
print('SW execution time: ',sw_exec_time)

start
SW execution time:  100.12047100067139


In [45]:
from pynq import Xlnk
import pynq.lib.dma
# Allocate buffers for the input and output signals
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(1024*2048,), dtype=np.int32)
out_buffer = xlnk.cma_array(shape=(1024*1024,), dtype=np.int32)

a=np.ones(1024*2048, dtype=np.int32)
# Copy the samples to the in_buffer
np.copyto(in_buffer,a)
print(in_buffer)
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
print("start")
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
print('Hardware execution time: ',hw_exec_time)
print('Hardware acceleration factor: ',sw_exec_time / hw_exec_time)

# Free the buffers
in_buffer.close()
out_buffer.close()

[1 1 1 ..., 1 1 1]


RuntimeError: DMA channel not idle